In [1]:
using JuMP
using Gurobi
using MixedIntegerExperiments
using Polyhedra
using CDDLib
using Plots
using DrakeVisualizer
using AxisArrays
using BenchmarkTools

In [2]:
# Pkg.clone("https://github.com/andreasnoack/LinearAlgebra.jl")
# Pkg.clone("git@github.com:JuliaPolyhedra/ConvexHull.jl.git")

In [3]:
# Pkg.checkout("CDDLib", "eltypeit")
# Pkg.checkout("Polyhedra")

In [4]:
verbose = false;
show_plots = true;

In [5]:
# parameters
nsteps = 10
θmax = 1.5
h = 0.1
I = 1.
m = 10.
g = [0; -9.81]
r0 = [0.5; 0.8]
ṙ0 = [0.1; 0.0]
θ0 = 0.
ω0 = 0.
max_contact_point_accel = 15.
environment = [
    contact_region([0.; 0.], [1.; 0.], 0.8, 2 * m * norm(g)); 
    axis_aligned_free_box_region([0.; 0.], [1.; 1.])];
contact_kinematic_regions = Dict( # TODO: θ dependence?
    :foot => SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 -1.; 0.5 -1.]),
    :hand => SimpleVRepresentation([-0.5 -0.5; 0.5 -0.5; -0.5 0.5; 0.5 0.5]));

In [6]:
coords = Axis{:coord}([:x, :z])
steps = Axis{:step}(1 : nsteps)
contacts = Axis{:contact}(collect(keys(contact_kinematic_regions)))
regions = Axis{:region}(1 : length(environment))
ncontacts = length(contacts)

2

In [27]:
if show_plots
    plot_environment(environment)
    for (i, region) in enumerate(environment)
        isfree(region) || plot_allowable_forces(region, i)
    end
    plot_kinematic_regions(contact_kinematic_regions)
end

In [8]:
model = Model(solver = GurobiSolver(Presolve = 0, OutputFlag = Int(verbose)));

In [9]:
rs = @axis_variables(model, r, coords, steps)
θs = @axis_variables(model, θ, steps)
ṙs = @axis_variables(model, ṙ, coords, steps)
ωs = @axis_variables(model, ω, steps)
rcs = @axis_variables(model, rc, coords, contacts, steps)
ṙcs = @axis_variables(model, ṙc, coords, contacts, steps)
fs = @axis_variables(model, f, coords, contacts, steps)
Fs = @axis_variables(model, F, coords, steps)
Ts = @axis_variables(model, T, steps)
ss = @axis_variables(model, s, coords, contacts, steps)
wzxs = @axis_variables(model, wzx, contacts, steps)
wxzs = @axis_variables(model, wxz, contacts, steps)
γs = @axis_variables(model, γ, contacts, regions, steps(1 : length(steps) - 1))
Γs = @axis_variables(model, Γ, contacts, regions);

In [10]:
# 4.6
zs = AxisArray(Array{Variable, 3}(ncontacts, length(environment), nsteps), contacts, regions, steps)
product_regions = collect(region.position * region.force for region in environment)
for i = 1 : ncontacts, n = 1 : nsteps
    rc = rcs[contacts(i), steps(n)]
    f = fs[contacts(i), steps(n)]
    zs[contacts(i), steps(n)] = hull_reformulation(model, product_regions, [rc; f])
end

In [11]:
# contact point dynamics
for i = 1 : ncontacts, n = 1 : nsteps - 1
    @constraints(model, begin
        rcs[contacts(i), steps(n + 1)] .== rcs[contacts(i), steps(n)] + h * ṙcs[contacts(i), steps(n + 1)]
        -h * max_contact_point_accel .<= ṙcs[contacts(i), steps(n + 1)] - ṙcs[contacts(i), steps(n)]
        ṙcs[contacts(i), steps(n + 1)] - ṙcs[contacts(i), steps(n)] .<= h * max_contact_point_accel
    end)
end

In [12]:
# (4.13)
for i = 1 : ncontacts, n = 1 : nsteps
    s = ss[contacts(i), steps(n)]
    rc = rcs[contacts(i), steps(n)]
    r = rs[steps(n)]
    @constraint(model, s .== rc - r)
end

In [13]:
# 4.18
for n = 1 : nsteps - 1
    @constraints(model, begin
        rs[steps(n + 1)] .== rs[steps(n)] + h * ṙs[steps(n + 1)]
        θs[steps(n + 1)] == θs[steps(n)] + h * ωs[steps(n + 1)]
        ṙs[steps(n + 1)] .== ṙs[steps(n)] + h / m * Fs[steps(n + 1)]
        ωs[steps(n + 1)] == ωs[steps(n)] + h / I * Ts[steps(n + 1)]
    end)
end

for n = 1 : nsteps
    @constraints(model, begin
        Fs[steps(n)] .== m * g + sum(fs[steps(n), contacts(i)] for i = 1 : length(contacts))
        Ts[steps(n)] == sum(wzxs[steps(n), contacts(i)] - wxzs[steps(n), contacts(i)] for i = 1 : length(contacts))
    end)
end

In [14]:
# convex bilinear term approximations (4.12, 4.13)
for i = 1 : ncontacts, n = 1 : nsteps
    # TODO: ranges, piecewise
    wxz = wxzs[contacts(i), steps(n)]
    wzx = wzxs[contacts(i), steps(n)]
    s = ss[contacts(i), steps(n)]
    f = fs[contacts(i), steps(n)]
    mccormick_envelope_constraints(model, wxz, s[:x], f[:z], -100., 100., -100., 100.)
    mccormick_envelope_constraints(model, wzx, s[:z], f[:x], -100., 100., -100., 100.)
end

In [15]:
# initial conditions
@constraints(model, begin
    rs[steps(1)] .== r0
    θs[steps(1)] == θ0
    ṙs[steps(1)] .== ṙ0
    ωs[steps(1)] == ω0
end)

# final conditions
@constraints(model, begin
    ṙs[steps(length(steps))] .== 0
    ωs[steps(length(steps))] == 0
end)

In [18]:
# kinematic constraints

# angle
@constraints(model, begin
    -θmax .<= θs
    θs .<= θmax
end)

# contact point positions relative to CoM
# NOTE: adding these increased solve time a lot
for contact in keys(contact_kinematic_regions), n = 1 : nsteps
    s = ss[contacts(contact), steps(n)]
    polyhedron_constraints(model, contact_kinematic_regions[contact], s)
end

In [19]:
# section 4.4
# objective function + slack parameterization
@constraint(model, Γs .== sum(γs[steps(n)] for n = 1 : length(steps) - 1));
for n = 1 : length(steps) - 1
    @constraints(model, begin
        -γs[steps(n)] .<= zs[steps(n + 1)] - zs[steps(n)]
        zs[steps(n + 1)] - zs[steps(n)] .<= γs[steps(n)]
        0 .<= γs
        γs .<= 1
    end)
end

@objective(model, Min, sum(Γ) + vecdot(fs, fs) + vecdot(ṙcs, ṙcs) + vecdot(θs, θs));

In [20]:
@benchmark solve($model)

BenchmarkTools.Trial: 
  memory estimate:  142.03 KiB
  allocs estimate:  73
  --------------
  minimum time:     31.023 ms (0.00% GC)
  median time:      32.802 ms (0.00% GC)
  mean time:        33.442 ms (0.00% GC)
  maximum time:     50.947 ms (0.00% GC)
  --------------
  samples:          150
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [21]:
plot(getvalue.(rs[:x]), lab = "x")

In [22]:
plot(getvalue.(rs[:z]), lab = "z")

In [23]:
plot(getvalue.(θs), lab = "θ")

In [24]:
# vis = MixedIntegerExperiments.plot_piecewise_mccormick(20, -1., 1., -1., 1.);

In [25]:
using Base.Test
# TODO: more checks on the final result

zvals = similar(zs, Float64)
for i = 1 : length(contacts), j = 1 : length(steps)
    @test sum(getvalue(zs[contacts(i), steps(j)])) == 1
end